In [19]:
import numpy as np
from dataclasses import dataclass
from functools import cache

@dataclass
class Piece():
  rows: tuple[tuple[bool]]
  columns: tuple[tuple[bool]]

  def __repr__(self):
    top = " " + " ".join('-' if row else ' ' for row in self.rows[0])
    first = " ".join('|' if col else ' ' for col in self.columns[0])
    middle = " " + " ".join('-' if row else ' ' for row in self.rows[1])
    second = " ".join('|' if col else ' ' for col in self.columns[1])
    bottom = " " + " ".join('-' if row else ' ' for row in self.rows[2])
    return "\n".join((top, first, middle, second, bottom))
  
  def __hash__(self):
      return hash((self.rows, self.columns))

  def __eq__(self, other):
      if isinstance(other, Piece):
          return self.rows == other.rows and self.columns == other.columns
      return NotImplemented
    
  def rotated(self):
    rows = np.rot90(self.columns, 1)
    columns = np.rot90(self.rows, 1)

    while all(not x for x in rows[0]) and all(not y for y in columns[0]):
      # top row is empty, so we can shift digit up
      rows = np.roll(rows, -1, 0)
      columns = np.roll(columns, -1, 0)

    return Piece(rows=tuple(map(tuple, rows)), columns=tuple(map(tuple, columns)))
  
  @cache
  def rotations(self):
    a = self.rotated()
    b = a.rotated()
    c = b.rotated()
    d = c.rotated()
    return set((a, b, c, d))

pieces = [
  Piece(rows=((True, False), (True, False), (False, False)), columns=((True, True, False), (False, False, False))),
  Piece(rows=((False, False), (False, False), (False, False)), columns=((True, False, False), (True, False, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((False, True, False), (True, False, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((False, True, False), (False, True, False))),
  Piece(rows=((False, False), (True, False), (False, False)), columns=((True, True, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, False, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, False, False), (True, True, False))),
  Piece(rows=((True, False), (False, False), (False, False)), columns=((False, True, False), (False, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, True, False), (True, True, False))),
  Piece(rows=((True, False), (True, False), (True, False)), columns=((True, True, False), (False, True, False))),
]

In [20]:
pieces[9].rotations()

{   -
 | | |
  - -
      
     ,
  -  
 |    
  -  
 | |  
  -  ,
  -  
 | |  
  -  
   |  
  -  ,
  - -
 | | |
  -  
      
     }

In [21]:
pieces[1].rotations()

{    
 |    
     
 |    
     ,
  - -
      
     
      
     }

In [22]:
from marshal import loads, dumps

class Puzzle():
  rows: list[list[bool]]
  cols: list[list[bool]]

  def __init__(
      self,
      rows=[[False for _ in range(5)] for _ in range(5)],
      cols=[[False for _ in range(6)] for _ in range(4)],
    ):
    self.rows = rows
    self.cols = cols

  def __repr__(self):
    text = ""
    for (row, col) in zip(self.rows, self.cols):
      text += " "
      text += " ".join('-' if row_piece else '.' for row_piece in row)
      text += "\n"
      text += " ".join('|' if col_piece else '.' for col_piece in col)
      text += "\n"
    text += " "
    text += " ".join('-' if row_piece else '.' for row_piece in self.rows[-1])
    return text

  def __hash__(self):
      return hash((tuple(map(tuple, self.rows)), tuple(map(tuple, self.cols))))

  def __eq__(self, other):
      if isinstance(other, Puzzle):
          return self.rows == other.rows and self.cols == other.cols
      return NotImplemented
  
  @cache
  def with_piece(self, piece: Piece, toprow: int, topcol: int) -> "Puzzle":
    clone = Puzzle(loads(dumps(self.rows)), loads(dumps(self.cols)) )

    for row in range(len(piece.rows)):
      for col in range(len(piece.rows[row])):
        # order matters
        if piece.rows[row][col]:
          if (toprow+row >= len(clone.rows) or topcol+col >= len(clone.rows[toprow+row])):
            return None
          if clone.rows[toprow+row][topcol+col]:
            return None
          clone.rows[toprow+row][topcol+col] = True

    for row in range(len(piece.columns)):
      for col in range(len(piece.columns[row])):
        # order matters
        if piece.columns[row][col]:
          if (toprow+row >= len(clone.cols) or topcol+col >= len(clone.cols[toprow+row])):
            return None
          if clone.cols[toprow+row][topcol+col]:
            return None
          clone.cols[toprow+row][topcol+col] = True

    return clone
  
  def spaces_left(self):
    return np.size(self.cols) - np.count_nonzero(self.cols) + np.size(self.rows) - np.count_nonzero(self.rows)

puzzle = Puzzle()

In [23]:
%timeit Puzzle().with_piece(pieces[7], 3, 4)

6.13 µs ± 1.85 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [24]:
Puzzle()

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [25]:
puzzle.spaces_left()

49

In [26]:
puzzle.with_piece(pieces[0], 1, 3)

 . . . . .
. . . . . .
 . . . - .
. . . | | .
 . . . - .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [27]:
puzzle.with_piece(pieces[7].rotated(), 0, 0)

 - - . . .
| . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [28]:
puzzle.with_piece(pieces[3], 0, 0).with_piece(pieces[4].rotated(), 0, 1)

 - - - . .
. | | . . .
 - - . . .
. | . . . .
 - . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [29]:
Puzzle().with_piece(pieces[9].rotated().rotated().rotated(), 1, 0)

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [30]:
from copy import copy

bad_puzzles = set()

def attempt(puzzle: Puzzle, used_pieces: tuple):
  if puzzle in bad_puzzles:
    return None
  if puzzle.spaces_left() == 2:
    return puzzle, used_pieces
  
  candidates = []
  for i in range(len(pieces)):
    if i in [j for (j, _, _, _) in used_pieces]:
      continue

    for rotation in pieces[i].rotations():
      for row in range(0, len(puzzle.rows)):
        for col in range(0, len(puzzle.cols)):          
          candidate = puzzle.with_piece(rotation, row, col)
      
          if candidate is not None:
            if candidate in bad_puzzles:
              continue

            candidate_used_pieces = copy(used_pieces)
            candidate_used_pieces.append((i, rotation, row, col))
            candidates.append((candidate, candidate_used_pieces))
    
  for candidate, candidate_used_pieces in candidates:
    result = attempt(candidate, candidate_used_pieces)
    if result is not None:
      return result

  bad_puzzles.add(puzzle)
  return None

# pick a puzzle piece
# choose rotation
# find top-left free corner
# put piece if it fits
# check for solved
rot9 = pieces[9].rotated().rotated().rotated()
puzzle = Puzzle().with_piece(rot9, 1, 0)
attempted = attempt(puzzle, [(9, rot9, 1, 0)])
attempted

MemoryError: 

In [ ]:
len(bad_puzzles)

197162

In [ ]:
next(iter(bad_puzzles))

 - - . - .
. | | | | .
 - - - - .
| | | . . .
 - - - . .
| . | | | .
 . - - - .
| | . . . .
 . . . . .